## 案例2：使用 Embedding 进行知识搜索

## 使用嵌入的文本搜索相似度最高的评论内容 Get_embeddings_from_dataset.ipynb

In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval

datafile_path = "data/fine_food_reviews_with_embeddings_1k.csv"
#1.读取知识库中的embedding列
df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(literal_eval).apply(np.array)


In [3]:
from utils.embedings_utils import get_embedding,cosine_similarity

#2.定义函数，根据字符，查找评论中相似度最高的3条记录
def search_reviews(df, product_description, n=3, pprint=True):
    #1.将查询的字符串，调用embedding模型，进行embedding化
    product_embedding = get_embedding(
        product_description,
        model="text-embedding-3-small"
    )
    #2.根据查询字符串的embedding值，与embedding列，进行相似度计算,放入到新增列similarity
    df["similarity"] = df.embedding.apply(lambda x: cosine_similarity(x, product_embedding))

    #3.对similarity列进行排序，取出前3条,输出标题和内容
    results = (
        df.sort_values("similarity", ascending=False)
        .head(n)
        .combined.str.replace("Title: ", "")
        .str.replace("; Content:", ": ")
    )
    if pprint:
        for r in results:
            print(r[:200])
            print()
    return results

#3.调用测试
results = search_reviews(df, "delicious beans", n=3)


Delicious!:  I enjoy this white beans seasoning, it gives a rich flavor to the beans I just love it, my mother in law didn't know about this Zatarain's brand and now she is traying different seasoning

Fantastic Instant Refried beans:  Fantastic Instant Refried Beans have been a staple for my family now for nearly 20 years.  All 7 of us love it and my grown kids are passing on the tradition.

Delicious:  While there may be better coffee beans available, this is my first purchase and my first time grinding my own beans.  I read several reviews before purchasing this brand, and am extremely 



In [4]:
results = search_reviews(df, "whole wheat pasta", n=3)


Tasty and Quick Pasta:  Barilla Whole Grain Fusilli with Vegetable Marinara is tasty and has an excellent chunky vegetable marinara.  I just wish there was more of it.  If you aren't starving or on a 

sooo good:  tastes so good. Worth the money. My boyfriend hates wheat pasta and LOVES this. cooks fast tastes great.I love this brand and started buying more of their pastas. Bulk is best.

Bland and vaguely gamy tasting, skip this one:  As far as prepared dinner kits go, "Barilla Whole Grain Mezze Penne with Tomato and Basil Sauce" just did not do it for me...and this is coming from a p



We can search through these reviews easily. To speed up computation, we can use a special algorithm, aimed at faster search through embeddings.

In [5]:
results = search_reviews(df, "bad delivery", n=1)


great product, poor delivery:  The coffee is excellent and I am a repeat buyer.  Problem this time was with the UPS delivery.  They left the box in front of my garage door in the middle of the drivewa



As we can see, this can immediately deliver a lot of value. In this example we show being able to quickly find the examples of delivery failures.

In [6]:
results = search_reviews(df, "spoilt", n=1)


Disappointed:  The metal cover has severely disformed. And most of the cookies inside have been crushed into small pieces. Shopping experience is awful. I'll never buy it online again.



In [7]:
results = search_reviews(df, "pet food", n=2)


Great food!:  I wanted a food for a a dog with skin problems. His skin greatly improved with the switch, though he still itches some.  He loves the food. No recalls, American made with American ingred

Great food!:  I wanted a food for a a dog with skin problems. His skin greatly improved with the switch, though he still itches some.  He loves the food. No recalls, American made with American ingred

